In [30]:
%pip install simplejson


[notice] A new release of pip is available: 23.0 -> 23.1.2
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [31]:
# CONNECT TO REDDIT THROUGH UAUTH

import requests

# developer account info
client_id = "iBUlXklHFr1KDzhJ7BVdAw"
secret = "9xDTwhK5AkhCKm27i0uM9UcQcH_DjQ"
uname = "hide-it-AI4G"
pw = "AI4GoodLab"

  # nrequest OAuth token
auth = requests.auth.HTTPBasicAuth(client_id, secret)

# login method (username & pw)
data = {'grant_type': 'password', 'username': uname, 'password': pw}

# setup header info (brief description of the app from the github)
headers = {'User-Agent': 'Hide-It: https://github.com/Rain1618/Hide-It (use for training a model in the AI4Good Lab, Montreal cohort)'}

# send request for an OAuth token
res = requests.post('https://www.reddit.com/api/v1/access_token', auth=auth, data=data, headers=headers)

# convert response to JSON and pull access_token value
TOKEN = res.json()['access_token']

# add authorization to headers dictionary
headers = {**headers, **{'Authorization': f"bearer {TOKEN}"}}

# while the token is valid (~2 hours) add headers=headers to requests
request = requests.get('https://oauth.reddit.com/api/v1/me', headers=headers)


In [129]:
# SETUP TRIGGER CATEGORIES

# target subreddits
ed_subreddits = ['EatingDisorder', 
                 'AnorexiaNervosa', 
                 'Bulimia', 
                 'Eating_disorders', 
                 'BingeEatingDisorder', 
                 'EDAnonymous'
                ]
abuse_subreddits = ['domesticviolence',
                                'NarcissisticAbuse',
                                'TrueNarcissisticAbuse',
                                'abusesurvivors',
                                'abusiverelationships',
                                'emotionalabuse'
                               ]
addiction_subreddits = ['alcoholism',
                        'addiction',
                        'opiatesRecovery',
                        'stopdrinking',
                        'stopsmoking',
                        'stopspeeding'
                       ]
suicide_subreddits = ['suicideWatch',
                      'TimeToGo',
                      'SuicideBereavement',
                     ]
sexual_violence_subreddits = ['SexualAssault',
                             'AdultSurvivors',
                             'rape',
                            ]

safe_subreddits = ['CasualConversations',
                  'UpliftingNews']

trigger_categories = {"eating disorders":ed_subreddits, "abuse":abuse_subreddits, 
                      "addiction":addiction_subreddits, "suicide":suicide_subreddits, 
                      "sexual violence":sexual_violence_subreddits, "safe":safe_subreddits}

target_info = ['title', 'selftext', 'subreddit', 'name']
  

In [141]:
# DOWNLOAD DATA FROM API TO CSV

import pandas as pd
import time

for cat_name,cat_subs in trigger_categories.items():
    
    data = pd.DataFrame(columns = ['title', 'selftext', 'subreddit', 'name', 'trigger'])
    limit = 10
    search_after = None
    flair_name = None
    
    for sub in cat_subs:
        
        print(sub)
        count = 0
        search_after = None

        while (count < 20):

            url  = "https://oauth.reddit.com/r/" + sub + "/new.json"
            res = requests.get(url, headers=headers, params={'limit': limit,'after': search_after})
            count = count+limit
            posts = res.json()['data']['children']
            
            print(len(posts))

            for post in posts:
                
                post_data = {key: post['data'][key] for key in target_info}
                post_data['trigger'] = cat_name
                data = data.append(post_data, ignore_index=True)

            # if close to going over request limit, sleep for time remaining until reset

            if float(res.headers['X-Ratelimit-Remaining']) < 5:
                time.sleep(float(res.headers['X-Ratelimit-Reset']))

            # save the last post as the index to search before, otherwise will get duplicates on iterations
            search_after = data['name'].iloc[len(data) - 1]

    # save data from this subreddit
    data.to_csv(cat_name + "_data.csv", encoding='utf-8')
    print(len(data))
    

EatingDisorder
10
10
AnorexiaNervosa
10
10
Bulimia
10
10
Eating_disorders
10
10
BingeEatingDisorder
10
10
EDAnonymous


KeyboardInterrupt: 